<a href="https://colab.research.google.com/github/filipenobre28/TelecomX/blob/main/DesfaioTelecomX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dados_principal = ('/content/TelecomX_Data.json')

In [ ]:
dados_principal.fillna(0, inplace=True)

In [ ]:
dados_principal = pd.read_json(dados_principal)

In [ ]:
dados_principal.head()

In [ ]:
dados_hashable = dados_principal.copy()

for col in dados_hashable.columns:
  if any(isinstance(x, ( dict, list)) for x in dados_hashable[col]):
    dados_hashable[col] = dados_hashable[col].astype(str)



In [ ]:
print(dados_principal.dtypes)

In [ ]:
dados_principal.head()

In [ ]:
print(f"dados_principla: {dados_hashable.duplicated().sum()}")

In [ ]:
!pip install ydata-profiling

In [ ]:
from ydata_profiling import ProfileReport
profile_report = ProfileReport(dados_principal)
profile_report.to_file("relatorio.html")

In [ ]:
profile_report.to_notebook_iframe()

In [ ]:
dados_principal = dados_principal.drop(columns=['customerID'])

In [ ]:
import json


def parse_phone(x):
  if isinstance(x, str):
    try:
        return json.loads(x.replace("'", "\""))
    except json.JSONDecodeError:
          return None
  elif isinstance(x, dict):
        return x
  return None



dados_principal['phone'] = dados_principal['phone'].apply(parse_phone)
phone_desaninhado = pd.json_normalize(dados_principal['phone'])
dados_principal = pd.concat([dados_principal, phone_desaninhado], axis=1)
dados_principal = dados_principal.drop(columns=['phone'])

In [ ]:
def parse_phone(x):
    if isinstance(x, str):
        try:
            return json.loads(x.replace("'", "\""))
        except json.JSONDecodeError:
            return None
    elif isinstance(x, dict):
        return x
    return None

dados_principal['internet'] = dados_principal['internet'].apply(parse_phone)

phone_desaninhado = pd.json_normalize(dados_principal['internet'])
dados_principal = pd.concat([dados_principal.drop('internet', axis=1), phone_desaninhado], axis=1)

In [ ]:
def parse_phone(x):
    if isinstance(x, str):
        try:
            return json.loads(x.replace("'", "\""))
        except json.JSONDecodeError:
            return None
    elif isinstance(x, dict):
        return x
    return None


dados_principal['account'] = dados_principal['account'].apply(parse_phone)


phone_desaninhado = pd.json_normalize(dados_principal['account'])
dados_principal = pd.concat([dados_principal.drop('account', axis=1), phone_desaninhado], axis=1)

In [ ]:
dados_principal.head()

In [ ]:
profile = ProfileReport(dados_principal, title="Ratorio dos dados")
profile.to_notebook_iframe()

In [ ]:
for col in dados_principal.columns:
  if dados_principal[col].apply(lambda x: isinstance(x, dict,)).any():
    print(f"Coluna '{col}' Contem dicionario ")

In [ ]:
print(dados_principal.dtypes)

In [ ]:
customer_expandido = dados_principal['customer'].apply(pd.Series)
customer_expandido = customer_expandido.add_prefix('customer_')

dados_principal = dados_principal.drop(columns=['customer'] )
dados_principal = pd.concat([dados_principal, customer_expandido], axis=1)


In [ ]:
dados_limpo = dados_principal.drop_duplicates()
print(f"Total de apos limpeza: {len(dados_limpo)}")

In [ ]:
dados_principal = dados_principal.loc[:, ~dados_principal.columns.duplicated()]

In [ ]:
dados_principal.head()

In [ ]:
colunas_binarias = ['Churn', 'customer_Partner', 'customer_Dependents', 'PhoneService',
                    'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                    'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']
dados_principal[colunas_binarias] = dados_principal[colunas_binarias].replace({'Yes' : 1, 'No' : 0})

In [ ]:
dados_principal.head()

In [ ]:
colunas_pt = {
    'Churn': 'Evasao',
    'PhoneService': 'Servico_Telefone',
    'MultipleLines': 'Multiplas_Linhas',
    'InternetService': 'Tipo_Internet',
    'OnlineSecurity': 'Seguranca_Online',
    'OnlineBackup': 'Backup_Online',
    'DeviceProtection': 'Protecao_Dispositivo',
    'TechSupport': 'Suporte_Tecnico',
    'StreamingTV': 'Streaming_TV',
    'StreamingMovies': 'Streaming_Filmes',
    'Contract': 'Tipo_Contrato',
    'PaperlessBilling': 'Fatura_Digital',
    'PaymentMethod': 'Metodo_Pagamento',
    'Charges.Monthly': 'Cobranca_Mensal',
    'Charges.Total': 'Cobranca_Total',
    'customer_gender': 'Genero',
    'customer_SeniorCitizen': 'Idoso',
    'customer_Partner': 'Possui_Conjuge',
    'customer_Dependents': 'Possui_Dependentes',
    'customer_tenure': 'Meses_Contrato'
}

dados_principal = dados_principal.rename(columns=colunas_pt)

print("Colunas renomeadas:")
print(dados_principal.columns)


In [ ]:
descricao_numerica = dados_principal.describe().T[['min', 'max', 'std', 'mean', '25%', '50%', '75%', 'count']]

descricao_numerica = descricao_numerica.rename(columns={'50%' :  'Mediana', 'min' : 'Minimo', 'max': 'Maximo', 'std' : 'Desvio Padrao', 'mean' : 'Media',  '25%' : 'Primeiro Quartil',
'75%' : 'Terceiro Quartil', 'count' : 'Contagem' })
print('Estatistica numericas: ')
print(descricao_numerica)

In [ ]:
print(dados_principal.dtypes)

In [ ]:
print(dados_principal.columns)

In [ ]:
!pip install kaleido

In [ ]:
import pandas as pd
import plotly.express as px
from IPython.display import display, Image, HTML

proporcao_evasao = dados_principal['Evasao'].value_counts(normalize=True)

labels_map = {1: 'Evasão', 0: 'Permaneceu'}

fig = px.pie(
    names=proporcao_evasao.index.map(labels_map),
    values=proporcao_evasao.values,
    title='📉 Proporção de Clientes por Evasão',
    hole=0.3,
    color_discrete_sequence=['#ff6b6b', '#51cf66']
)

fig.update_traces(textinfo='percent+label')
fig.update_layout(legend_title_text='Status do Cliente')

fig.show()


fig.write_image("proporcao_evasao.png")

In [ ]:
df = dados_principal.copy()

df['Evasao'] = df['Evasao'].map({1: 'Evadido', 0: 'Permaneceu'})
df['Evasao'] = df['Evasao'].fillna('Desconhecido')

variaveis_categoricas = ['Metodo_Pagamento', 'Tipo_Contrato', 'Idoso']
nomes_colunas = {
    'Metodo_Pagamento': 'Método de Pagamento',
    'Tipo_Contrato': 'Tipo de Contrato',
    'Idoso': 'Cliente Idoso'
}

mapa_cores = {
    'Permaneceu': '#1f77b4',
    'Evadido': '#ff7f0e',
    'Desconhecido': '#cccccc'
}

for coluna in variaveis_categoricas:
    df_plot = df.copy()
    tabela_evasao = pd.crosstab(df_plot[coluna], df_plot['Evasao'], normalize='index') * 100

    print(f"\nDistribuição de Evasão por {nomes_colunas.get(coluna, coluna)}:")
    print(tabela_evasao.to_markdown(numalign="left", stralign="left"))

    tabela_evasao = tabela_evasao.reset_index()
    colunas_evasao = [c for c in ['Permaneceu', 'Evadido', 'Desconhecido'] if c in tabela_evasao.columns]

    df_melted = tabela_evasao.melt(
        id_vars=[coluna],
        value_vars=colunas_evasao,
        var_name='Status de Evasão',
        value_name='Proporção de Clientes (%)'
    )

    fig = px.bar(
        df_melted,
        x=coluna,
        y='Proporção de Clientes (%)',
        color='Status de Evasão',
        title=f'📊 Proporção de Evasão por {nomes_colunas.get(coluna, coluna)}',
        labels={
            coluna: nomes_colunas.get(coluna, coluna),
            'Proporção de Clientes (%)': 'Proporção (%)'
        },
        barmode='stack',
        color_discrete_map=mapa_cores,
        category_orders={"Status de Evasão": ['Permaneceu', 'Desconhecido', 'Evadido']}
    )

    fig.update_layout(legend_title_text='Status do Cliente')


    fig.show()

    nome_arquivo = f"evasao_por_{coluna.lower()}.png"
    fig.write_image(nome_arquivo, scale=2)
    print(f"Gráfico salvo como: {nome_arquivo}")

-----

# Relatório Resumido de Análise de Evasão de Clientes

## Introdução

Este relatório apresenta uma análise concisa do fenômeno da **evasão de clientes (Churn)** em uma empresa de telecomunicações. O objetivo principal é identificar os fatores-chave que impulsionam a saída de clientes e oferecer recomendações estratégicas para melhorar a retenção.

-----

## Processamento de Dados

Os dados foram importados de um arquivo JSON e submetidos a um processo de limpeza e transformação. As principais etapas incluíram:

  * **Desaninhamento de colunas JSON:** Informações aninhadas em "phone", "internet", "account" e "customer" foram extraídas e transformadas em colunas independentes para facilitar a análise.
  * **Padronização de dados:** Valores binários ('Yes'/'No') foram convertidos para 1 e 0.
  * **Renomeação de colunas:** As colunas foram renomeadas para o português, tornando o dataset mais intuitivo (ex: 'Churn' para 'Evasao', 'customer\_gender' para 'Genero').

-----

## Principais Descobertas e Insights

A análise exploratória dos dados revelou padrões importantes:

 **Taxa de Evasão:** A empresa enfrenta uma taxa de evasão de 26.54%, o que é um desafio considerável.



**Impacto do Método de Pagamento:** Clientes que utilizam Cheque Eletrônico apresentam a maior taxa de evasão (45.18%), muito superior aos outros métodos. Isso sugere problemas específicos com essa forma de pagamento.



**Influência do Tipo de Contrato:** Existe uma forte correlação entre o tipo de contrato e a evasão. Clientes com contratos mensais ('Month-to-month') têm uma taxa de evasão significativamente alta (42.71%), enquanto contratos de um ou dois anos demonstram maior lealdade (11.26% e 2.84% de evasão, respectivamente).



**Perfil do Cliente Idoso:** Clientes idosos apresentam uma taxa de evasão mais elevada (41.89%) em comparação com clientes não idosos (23.61%), indicando que esse grupo pode ter necessidades ou desafios específicos não atendidos.



-----

## Recomendações

Com base nas descobertas, recomendamos as seguintes ações:

1.  **Reavaliar o Cheque Eletrônico:** Investigar e aprimorar a experiência de pagamento via Cheque Eletrônico, ou incentivar a migração para métodos mais estáveis, talvez com pequenos incentivos.
2.  **Incentivar Contratos de Longo Prazo:** Criar e promover programas de fidelidade e ofertas exclusivas para clientes que optam por contratos de um ou dois anos.
3.  **Atenção aos Clientes Idosos:** Desenvolver estratégias de suporte e comunicação adaptadas para clientes idosos, como suporte técnico simplificado ou materiais educativos focados em suas necessidades.
4.  **Monitoramento Contínuo:** Manter um acompanhamento constante das taxas de evasão e dos fatores de risco para ajustar as estratégias proativamente.

Ao focar nesses pontos, a empresa pode reduzir significativamente a taxa de churn e melhorar a satisfação e lealdade dos clientes.


-----

In [ ]:
dados_principal.to_json('Desafio_telecom.json', index=False)

In [ ]:
dados_principal.to_csv('Desafio_telecom.csv', index=False)